## Globals

In [1]:
#@title Imports
try:
  from google.colab import files
  IN_COLAB = True
except:
  IN_COLAB = False

import re
from math import prod

fread = lambda path: open(['inputs/',''][IN_COLAB]+path,'r').read()

class colors:
    PURPLE = '\033[95m'
    BLUE = '\033[94m'
    CYAN = '\033[96m'
    GREEN = '\033[92m'
    ORANGE = '\033[93m'
    RED = '\033[91m'
    WHITE = '\033[0m'
    BOLD = '\033[1m'

## Solutions

In [ ]:
#@title Day 17: Clumsy Crucible
from queue import PriorityQueue

input = fread('23-17-0.txt').splitlines()
graph = [list(map(int,l)) for l in input]

# Implementation of A* shamelessly stolen from redblobgames.com
def minimize_heat(g, start, goal):
  w = len(g[0])
  h = len(g)

  frontier = PriorityQueue()
  frontier.put((0, start, (0,0)))
  came_from = {start: None}
  cost_so_far = {start: 0}

  while not frontier.empty():
    _, current, d = frontier.get()
    if current == goal: break

    for dx,dy in {(1,0),(-1,0),(0,1),(0,-1)}-{d,(-d[0],-d[1])}:
      nexts = [(current[0]+mag*dx, current[1]+mag*dy) for mag in range(4)]
      for i,n in enumerate(nexts[1:]):
        nx,ny = (n[0], n[1])
        if 0 > nx or w <= nx or 0 > ny or h <= ny: break

        new_cost = cost_so_far[nexts[i]] + g[ny][nx]
        if (nx,ny) not in cost_so_far or new_cost < cost_so_far[(nx,ny)]:
          if (nx,ny) == (8,0): print('current',current,'\td',(dx,dy))
          cost_so_far[(nx,ny)] = new_cost
          priority = new_cost + goal[0] - nx + goal[1] - ny
          frontier.put((priority, (nx,ny), (dx,dy)))
          came_from[(nx,ny)] = nexts[i]

  curr = goal
  path = set()
  while curr in came_from:
    path = path.union({curr})
    curr = came_from[curr]

  for y in range(h):
    for x in range(w):
      if (x,y) in path: print(colors.BLUE,end='')
      print(g[y][x],end='')
      if (x,y) in path: print(colors.WHITE,end='')
    print()

  return cost_so_far[goal]

print(minimize_heat(graph,(0,0),(len(graph[0])-1,len(graph)-1)))

current (5, 0) 	d (1, 0)
2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533
114


In [ ]:
#@title Day 18: Lavaduct Lagoon
input = [l.split(' ') for l in fread('23-18.txt').splitlines()]

def calculate_area(insts, dirs):
  pts = [(0,0)]
  for l in insts:
    x,y = pts[-1]
    dx,dy = dirs[l[0]]
    mag = l[1]
    pts.append((x+mag*dx,y+mag*dy))

  # Pick's theorem:
  # A = i + b/2 - 1
  # Find i + b.

  # Shoelace theorem
  A = 0.5*abs(sum([pts[i+1][1]*pts[i][0] - pts[i+1][0]*pts[i][1] for i in range(len(pts)-1)]))

  # Count points on perimeter
  b = sum(abs(pt[0]-pts[i][0])+abs(pt[1]-pts[i][1]) for i,pt in enumerate(pts[1:]))

  # => A - b/2 + 1 = i
  # => A + b/2 + 1 = i + b
  return int(A + (0.5 * b) + 1)

data = [(l[0],int(l[1])) for l in input]
directions = {'U':(0,-1),'R':(1,0),'D':(0,1),'L':(-1,0)}
print(calculate_area(data, directions))

data = [re.sub('[()#]','',l[2]) for l in input]
data = [(l[-1],int(l[0:5],16)) for l in data]
directions = {'3':(0,-1),'0':(1,0),'1':(0,1),'2':(-1,0)}
print(calculate_area(data, directions))

92758
62762509300678


In [ ]:
#@title Day 19-1: Aplenty
import operator

f = fread('23-19.txt').split('\n\n')

OPERATORS = { '<': operator.lt, '>': operator.gt }
INDEXES = {'x': 0, 'm': 1, 'a': 2, 's': 3}

class Node:
  def __init__(self, name, rules):
    self.name = name
    self.rules = rules

  def parse(l):
    name = re.match('(\w+){',l).group()[:-1]

    rules = []
    rs = list(re.findall('{.*}',l)[0][1:-1].split(','))
    for r in rs[:-1]:
      idx, op, num, result = re.match('(\w+)([<>])(\d+):(\w+)',r).groups()
      rules.append(Node.__rule__(idx,op,num,result))
    rules.append(Node.__fallback__(rs[-1]))

    return Node(name, rules)

  def __rule__(idx, op, num, result):
    return lambda part: [None,result][OPERATORS[op](part[INDEXES[idx]],int(num))]

  def __fallback__(result):
    return lambda part: result

  def process(self, part) -> None | bool | str:
    for r in self.rules:
      result = r(part)
      if result != None: return result

    raise Exception('No rules defined')

def process(lookup_table, part):
  result = 'in'
  while result not in set('AR'):
    result = lookup_table[result].process(part)
  return result == 'A'

nodes = {Node.parse(l) for l in f[0].splitlines()}
lookup = {n.name: n for n in nodes}
parts = [tuple(map(int,re.match('{x=(\d+),m=(\d+),a=(\d+),s=(\d+)}',l).groups())) for l in f[1].splitlines()]
print(sum(sum(p) for p in parts if process(lookup, p)))


399284


In [ ]:
#@title Day 19-2: Aplenty
import operator
from math import prod

f = fread('23-19.txt').split('\n\n')

OPERATORS = { '<': operator.lt, '>': operator.gt }
INDEXES = {'x': 0, 'm': 1, 'a': 2, 's': 3}

class PartRange:
  def __init__(self, ranges={c: (1,4001) for c in 'xmas'}):
    self.ranges = ranges

  def split(self, key, op, num):
    # print('splitting',key,op,num,'with ranges',self.ranges)
    if op == '<':
      if self.ranges[key][1] <= num:
        print('returning self, None')
        return [self, None]

      if self.ranges[key][0] > num:
        print('returning None,Self')
        return [None, self]
      return [
        PartRange({**self.ranges, key: (self.ranges[key][0],num)}),
        PartRange({**self.ranges, key: (num, self.ranges[key][1])})
      ]

    if self.ranges[key][1] <= num: return [None, self]
    if self.ranges[key][0] > num: return [self, None]
    return [
      PartRange({**self.ranges, key: (num+1, self.ranges[key][1])}),
      PartRange({**self.ranges, key: (self.ranges[key][0],num+1)})
    ]

  def count(self):
    return prod(self.ranges[key][1] - self.ranges[key][0] for key in self.ranges)

  def __str__(self) -> str:
    return str(self.ranges)

  def __repr__(self) -> str:
    return str(self.ranges)

class Node:
  def __init__(self, rules):
    self.rules = rules

  def parse(l):
    rules = []
    conditions = list(re.findall('{.*}',l)[0][1:-1].split(','))
    for cond in conditions[:-1]:
      rules.append(Node.__rule__(*re.match('(\w+)([<>])(\d+):(\w+)',cond).groups()))
    rules.append(Node.__fallback__(conditions[-1]))
    return Node(rules)

  def split(self, part):
    curr = part
    results = []
    for r in self.rules:
      key,p1,p2 = r(curr)
      # print(curr,'=>',key,p1,p2)
      if p1 != None: results.append((key, p1))
      if p2 == None: break
      curr = p2
    return results

  def __rule__(idx, op, num, result) -> tuple:
    return lambda part: tuple([result] + part.split(idx, op, int(num)))

  def __fallback__(result) -> tuple:
    return lambda part: (result, part, None)

lookup = {l.split('{')[0]: Node.parse(l) for l in f[0].splitlines()}

total = 0
to_process = [('in', PartRange())]
while to_process:
  key,part = to_process[0]
  to_process = to_process[1:]
  # print(key,part)
  if key == 'A': total += part.count()
  if key not in set('AR'): to_process += lookup[key].split(part)
print(total)
#   if curr is int:
#     total += curr
#   else:
#     to_process.push(curr)



121964982771486


In [11]:
#@title Day 20: Pulse Propagation
from itertools import chain
from math import lcm
f = fread('23-20.txt').splitlines()

BMOD = 'broadcaster'

class Module:
  def __init__(self, name, *nexts):
    self.name = name
    self.nexts = list(n for n in nexts if n != None)

  def store(self, prev, signal): return True

  def __propagate__(self, override):
    return [(n, self.name, override) for n in self.nexts]

class Broadcaster(Module):
  def __init__(self, name, *nexts):
    super().__init__(name, *nexts)
    self.signal = False

  def store(self, prev, signal):
    self.signal = signal
    return True

  def process(self): return super().__propagate__(self.signal)

class FlipFlop(Module):
  def __init__(self, name, *nexts):
    super().__init__(name, *nexts)
    self.state = False

  def store(self, prev, signal): return not signal

  def process(self):
    self.state = not self.state
    return super().__propagate__(self.state)

class Conjunction(Module):
  def __init__(self, name, *nexts):
    super().__init__(name, *nexts)
    self.state = {}

  def store(self, prev, signal):
    self.state.update({prev: signal})
    return True

  def process(self): return super().__propagate__(self.on())
  def on(self): return any(not self.state[k] for k in self.state)

def parse(ls):
  ms = \
    [Broadcaster(*re.split('[ ->,]+', l)) for l in ls if re.match(BMOD,l)] \
    + [FlipFlop(*re.split('[% ->]+',l)[1:]) for l in ls if l[0] == '%'] \
    + [Conjunction(*re.split('[& ->]+',l)[1:]) for l in ls if l[0] == '&']

  lookup = {m.name: m for m in ms}

  for m in ms:
    for n in m.nexts:
      if n in lookup and type(lookup[n]) is Conjunction:
        lookup[n].store(m.name, False)

  return lookup

def push_button(ms, n_times):
  highs = 0
  lows = 0
  for i in range(n_times):
    step = [(BMOD, 'button', False)]
    while step:
      highs += sum(1 for s in step if s[2])
      lows += sum(1 for s in step if not s[2])
      to_process = [ms[curr[0]] for curr in step if curr[0] in ms and ms[curr[0]].store(*curr[1:])]
      step = [n for m in to_process for n in m.process()]
  return (lows,highs)

modules = parse(f)
print(prod(push_button(modules, 1000)))

def turn_on(ms):
  looking_for = {'sl','pq','rr','jz'}
  found = {}
  count = 0
  while True:
    count += 1
    step = [(BMOD, 'button', False)]
    while step:
      to_process = [ms[curr[0]] for curr in step if curr[0] in ms and ms[curr[0]].store(*curr[1:])]
      step = [n for m in to_process for n in m.process()]
      loop = {m for m in looking_for if not ms[m].on()}
      found.update({m: count for m in (loop - set(found))})
    if len(found) == len(looking_for): break

  return lcm(*(found[m] for m in found))

modules = parse(f)
print(turn_on(modules))

737679780
227411378431763


In [ ]:
#@title Day 21: Step Counter
from queue import PriorityQueue

f = fread('23-21.txt').splitlines()

def find_start(graph):
  for y,l in enumerate(graph):
    for x,c in enumerate(l):
      if c == 'S': return (x,y)

  return None

def search(graph, start, n_steps):
  w = len(graph[0])
  h = len(graph)
  step = {start}

  for i in range(n_steps):
    nexts = set()

    for current in step:
      for dx,dy in [(1,0),(-1,0),(0,1),(0,-1)]:
        next = (current[0]+dx,current[1]+dy)
        nx,ny = next
        if nx >= w or nx < 0 or ny >= h or ny < 0 or graph[ny][nx] == '#': continue
        nexts = nexts.union({next})

    step = nexts

  return len(step)

print(search(f, find_start(f), 64))

3743
